### Initialize

In [ ]:
import json
import os
import sys
from datetime import datetime

import pandas as pd
import requests
from ipywidgets import Dropdown

sys.path.append(os.path.join(sys.path[0], '..', '..'))
from demo.shared.tools import (TENANT_SUFFIX, StopExecution, get_base_url,
                               get_root_path, print_response)

base_url = get_base_url()
root_path = get_root_path()

### Select example which should be used

In [ ]:
example_path = os.path.join('..', 'examples')
options = next(os.walk(example_path))[1]
selected = Dropdown(
    options=options,
    description='Select:',
    value = None,
    disabled=False,
)
selected

### Create CSON (JSON-format) from CDS file (text format) using the CDS compiler

In [ ]:
if selected.value:
    cson_file_direct = os.path.join(root_path, '..', 'examples', selected.value, 'model.json')
    if os.path.exists(cson_file_direct):
        with open(cson_file_direct, encoding='utf-8') as f:
            cson = json.load(f)
    else:
        cds_file = os.path.join(root_path, '..', 'examples', selected.value, 'model.cds')
        cson_file = os.path.join(root_path, '..', 'shared', 'temp', f'{selected.value}.model.json')
        command_line_statement = (f'cds compile {cds_file} --to json -o {cson_file}')
        os.system(command_line_statement)
        with open(cson_file, encoding='utf-8') as f:
            cson = json.load(f)
        print(f'The following command line statement was executed to compile the example cds file to cson. Filename is temp{os.sep}{selected.value}.model.json):')
        print(command_line_statement)
    data_file = os.path.join(root_path, '..', 'examples', selected.value, 'data.json')
else:
    print('No example selected')

### List existing tenants

In [ ]:
r = requests.get(f'{base_url}/v1/tenant')
if r.status_code == 200:
    existing_tenants = [{k:datetime.fromisoformat(v) if k == 'createdAt' else v 
        for k, v in w.items()} for w in r.json()]
    if existing_tenants:
        df = pd.DataFrame.from_dict(existing_tenants)
        df.rename(columns={'name': 'Tenant Name', 'createdAt': 'Created At'}, inplace=True)
        display(df)
    else:
        print('Info: No tenants exist')
else:
    print(f'Error. HTTP response status code {r.status_code}')

### Create tenant

In [ ]:
r = requests.post(f'{base_url}/v1/tenant/{selected.value}{TENANT_SUFFIX}')
print_response(r)

### Check model consistency

In [ ]:
r = requests.post(f'{base_url}/v1/deploy/{selected.value}', json=cson\
                    , params={'simulate': True})
print_response(r)

### Deploy

In [ ]:
r = requests.post(f'{base_url}/v1/deploy/{selected.value}{TENANT_SUFFIX}', json=cson)
print_response(r)

### Load data

In [ ]:
if selected.value:
    data_file = os.path.join(root_path, '..', 'examples', selected.value, 'data.json')
    if os.path.exists(data_file):
        with open(data_file, encoding = 'utf-8') as f:
            data = json.load(f)
        r = requests.post(f'{base_url}/v1/data/{selected.value}{TENANT_SUFFIX}', json=data)
        if r.status_code == 200:
            print('Data successfully loaded')
        else:
            print(f'Error loading data. HTTP status code {r.status_code}')
            print(r.text)
    else:
        print('No data file exists')